### EDA
EDA Goes into further detail
* notebook/Project_EDA_001_001.ipynb
* notebook/Project_EDA_002_001.ipynb

### Notebook with model training

notebook/Project_Model_Training.ipynb

Inside data_ingestion.py - src/components/data_ingestion.py

```python


```